In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
path = "F:/for learn/Python/Building-Machine-Learning-Projects-with-TensorFlow-master/5/data/"

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data = pd.read_csv(path + "wine.csv")
feature_columns = ['Alcohol', 'Malic.acid', 'Ash', 'Acl', 'Mg', 'Phenols',
       'Flavanoids', 'Nonflavanoid.phenols', 'Proanth', 'Color.int', 'Hue',
       'OD', 'Proline']
data["Wine"] = data["Wine"] - 1  # original unique value [1, 2, 3]
data.head()

,Wine,Alcohol,Malic.acid,Ash,Acl,Mg,Phenols,Flavanoids,Nonflavanoid.phenols,Proanth,Color.int,Hue,OD,Proline
0,0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,0,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,0,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,0,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,0,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [3]:
data.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 14 columns):
Wine                    178 non-null int64
Alcohol                 178 non-null float64
Malic.acid              178 non-null float64
Ash                     178 non-null float64
Acl                     178 non-null float64
Mg                      178 non-null int64
Phenols                 178 non-null float64
Flavanoids              178 non-null float64
Nonflavanoid.phenols    178 non-null float64
Proanth                 178 non-null float64
Color.int               178 non-null float64
Hue                     178 non-null float64
OD                      178 non-null float64
Proline                 178 non-null int64
dtypes: float64(11), int64(3)
memory usage: 19.5 KB


In [4]:
X = data[feature_columns].values   # 178 * 13
y = data["Wine"].values

onc = OneHotEncoder(sparse=False)
Y = onc.fit_transform(y.reshape(-1, 1))  # 178 * 3

X, Y = shuffle(X, Y)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

In [5]:
epochs = 100
display_step = 10

In [6]:
tf.reset_default_graph()
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

X_ = tf.placeholder(dtype=tf.float32, shape=[None, 13])
Y_ = tf.placeholder(dtype=tf.float32, shape=[None, 3])
W = tf.Variable(tf.zeros(dtype=tf.float32, shape=[13, 3]))
b = tf.Variable(tf.zeros(dtype=tf.float32, shape=[1, 3]))

y_ = tf.nn.softmax(X_ @ W + b)
cross_entropy = tf.reduce_mean(-tf.reduce_sum(Y_ * tf.log(y_), axis=1))
acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(Y_, axis=1), tf.argmax(y_, axis=1)), 
                             dtype=tf.float32), 
                     axis=0)
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)

In [7]:
sess.run(tf.global_variables_initializer())
for epoch in range(epochs):
    sess.run(train_step, feed_dict={X_: X_train, Y_: Y_train})
    if epoch % display_step == 0 or epoch == epochs - 1:
        cost, accuracy = sess.run([cross_entropy, acc], feed_dict={X_: X_test, Y_: Y_test})
        print("epoch %d: test_cost: %.4f, test_acc: %.4f" % (epoch, cost, accuracy))

epoch 0: test_cost: 0.9179, test_acc: 0.9444
epoch 10: test_cost: 0.3364, test_acc: 0.9722
epoch 20: test_cost: 0.2270, test_acc: 0.9722
epoch 30: test_cost: 0.1830, test_acc: 0.9722
epoch 40: test_cost: 0.1593, test_acc: 0.9722
epoch 50: test_cost: 0.1443, test_acc: 0.9722
epoch 60: test_cost: 0.1339, test_acc: 0.9722
epoch 70: test_cost: 0.1262, test_acc: 0.9722
epoch 80: test_cost: 0.1203, test_acc: 0.9722
epoch 90: test_cost: 0.1155, test_acc: 0.9722
epoch 99: test_cost: 0.1119, test_acc: 0.9722
